In [1]:
import pandas as pd
import plotly.express as px
import os
import json
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time
import datetime

In [2]:
position_records = []

directory = os.fsencode('vehicle_positions_hist')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    with open(f'vehicle_positions_hist/{filename}', 'r') as f:
        new_records = json.load(f)
        for record in new_records:
            record['insert_ts'] = int(filename.replace('.json', ''))

    position_records.extend(new_records)

In [3]:
df = pd.DataFrame.from_records(position_records)

routes = pd.read_csv('routes.txt')

df = df.merge(routes[['route_id', 'route_color']], on='route_id', how='left')
df['route_color'] = df['route_color'].fillna('0076CE')
df['route_color'] = '#' + df['route_color']

df['vehicle_update_ts_rounded'] = (np.ceil(df['vehicle_update_ts'].fillna(0) / 30)).astype('int') * 30

In [4]:
rail_route_ids = ['101D', '101E', '101H', '103W', '107R', '109L', '113B', '113G', '117N', 'A', 'FF1', 'FF2', 'FF3', 'AB1']

df['size'] = 1
df.loc[df['route_id'].isin(rail_route_ids), 'size'] = 10

all_timestamps = df['vehicle_update_ts_rounded'].unique()

In [5]:
def get_filled_vehicle_df(vehicle_id):
    veh_df = df[df['vehicle_id'] == vehicle_id].sort_values('vehicle_update_ts_rounded').drop_duplicates()
    veh_df['row_num'] = veh_df.sort_values(['vehicle_update_ts_rounded','insert_ts'], ascending=[True,True]).groupby(['vehicle_update_ts_rounded']).cumcount() + 1
    veh_df = veh_df[veh_df['row_num'] == 1]

    veh_df = veh_df.set_index(veh_df['vehicle_update_ts_rounded'])
    veh_df.sort_index(inplace=True)

    new_index = pd.Index(np.arange(all_timestamps.min(), all_timestamps.max(), 30), name='vehicle_update_ts_rounded_full')
    veh_df = veh_df.reindex(new_index, method='pad').reset_index()

    veh_df = veh_df[veh_df['vehicle_update_ts_rounded_full'].between(veh_df['vehicle_update_ts_rounded'].min(), veh_df['vehicle_update_ts_rounded'].max())]

    return veh_df

In [6]:
filled_df = pd.concat([get_filled_vehicle_df(vehicle_id) for vehicle_id in df['vehicle_id'].unique()])

In [7]:
def export_image_smooth(timestamp):
    plot_df = filled_df[(filled_df['vehicle_update_ts_rounded_full'] == timestamp)]
    fig = px.scatter_mapbox(plot_df, lat='lat', lon='lon', height=1440, width=1440, center={'lat': 39.9, 'lon': -105}, zoom=10, color='route_color', color_discrete_map='identity', size='size', hover_data='vehicle_label')
    fig.update_layout(mapbox_style='carto-positron')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(showlegend=False)
    fig.update_layout(title={'text': datetime.datetime.fromtimestamp(timestamp).strftime('%B %d %Y, %-I:%M %p'), 'font': {'size': 50}, 'automargin': True, 'yref': 'paper'})
    fig.update_layout(title_x=0.5)
    fig.write_image(f'frames/{timestamp}.png')
    #return fig 

In [8]:
start_time = time.time()
prev_time = start_time
for i, timestamp in enumerate(filled_df['vehicle_update_ts_rounded_full'].unique()):    
    print(f'Exporting Frame {timestamp}...')

    export_image_smooth(timestamp)
    
    clear_output(wait=True)

    current_time = time.time()
    print(f'{i+1} / {len(all_timestamps)} done. {current_time - start_time} elapsed, {(current_time - start_time) / (i + 1) * (len(all_timestamps) - i - 1)} remaining')

3325 / 3190 done. 6098.669652462006 elapsed, -247.61515882176565 remaining
Exporting Frame 1718231700...


In [ ]:
os.system("ffmpeg -framerate 24 -pattern_type glob -i 'frames/*.png' -c:v mpeg4 -y -vb 40M movie.mp4")

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

0